In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.spatial import distance

In [ ]:
df=pd.read_csv("clustering.csv",header=None)
df = np.asarray(df)

In [ ]:
from sklearn.cluster import KMeans

def plot_kmeans_clusters(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)
    plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=30, cmap='viridis')

    centers = kmeans.cluster_centers_
    plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.75)
    plt.title('K-means clusters')
    plt.show()


In [ ]:
def count_clustering_scores(X, cluster_num, model, score_fun):
    # Napiszmy tę funkcje tak ogólnie, jak to możliwe. 
    # Zwróćcie uwagę na przekazanie obiektów typu callable: model i score_fun.
    if isinstance(cluster_num, int):
        cluster_num_iter = [cluster_num]
    else:
        cluster_num_iter = cluster_num
        
    scores = []    
    for k in cluster_num_iter:
        model_instance = model(n_clusters=k)
        labels = model_instance.fit_predict(X)
        wcss = score_fun(X, labels)
        scores.append(wcss)
    
    if isinstance(cluster_num, int):
        return scores[0]
    else:
        return scores

In [ ]:
from scipy.spatial import distance
import numpy as np


def min_interclust_dist(X, label):
    clusters = set(label)
    global_min_dist = np.inf
    for cluster_i in clusters:
        cluster_i_idx = np.where(label == cluster_i)
        for cluster_j in clusters:
            if cluster_i != cluster_j:
                cluster_j_idx = np.where(label == cluster_j)
                interclust_min_dist = np.min(distance.cdist(X[cluster_i_idx], X[cluster_j_idx]))
                global_min_dist = np.min([global_min_dist, interclust_min_dist])
    return global_min_dist


In [ ]:
plt.scatter(df[:,0], df[:,1])
plt.show()

# AgglomerativeClustering 

## Silhouette score

In [ ]:
from sklearn.metrics import silhouette_score
cluster_num_seq = range(2, 11) # Niektóre metryki nie działają gdy mamy tylko jeden klaster
silhouette_vec = count_clustering_scores(df, cluster_num_seq, AgglomerativeClustering, silhouette_score)
plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette_score')
plt.show()


5 lub 8 klastrów osiaga najwyższy sihoulette score

## 5 klastrów

In [ ]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=5)
y = model.fit_predict(df)
plt.scatter(df[:, 0], df[:, 1], c=y, s=30, cmap='viridis')
plt.show()

## 8 klastrów

In [ ]:
model = AgglomerativeClustering(n_clusters=8)
y = model.fit_predict(df)
plt.scatter(df[:, 0],df[:, 1], c=y, s=30, cmap='viridis')
plt.show()

## Podsumowanie

Wybierzmy 8 klastrów ponieważ ma większy silhouette score oraz rozdziela punkty które w 6 - klastrowaniu były fioletowe, pomimo widocznego podziału.

# KMeans

## min interclust dist

In [ ]:
cluster_num_seq = range(2, 12) # Niektóre metryki nie działają gdy mamy tylko jeden klaster
silhouette_vec = count_clustering_scores(df, cluster_num_seq, KMeans, min_interclust_dist)
plt.plot(cluster_num_seq, silhouette_vec, 'bx-')
plt.xlabel('k')
plt.ylabel('min_interclust_dist')
plt.show()

Największą minimalna odległość między klastrami mamy dla k = 3 lub 7 lub 8

## 3 klastrów

In [ ]:
model = KMeans(n_clusters=3, random_state=14)
y = model.fit_predict(df)
plt.scatter(df[:, 0], df[:, 1], c=y, s=30, cmap='viridis')
plt.show()

Zły wybór klastrów, fioletowy klaster jest podzielony i miesza się z żółtym

## 7 klastrów

In [ ]:
model = KMeans(n_clusters=7, random_state=1)
y = model.fit_predict(df)
plt.scatter(df[:, 0], df[:, 1], c=y, s=30, cmap='viridis')
plt.show()

## 8 klastrów

In [ ]:
model = KMeans(n_clusters=8, random_state=0)
y = model.fit_predict(df)
plt.scatter(df[:, 0], df[:, 1], c=y, s=30, cmap='viridis')
plt.show()

## Podsumowanie
Wybrałabym 7 klastrów ze zwględu na  min_interclust_dist oraz to że w przeciwnieństwie do 8 klastrów, zielony klaster nie jest podzielony. Zielony klaster ma spójny klształt "flagi" wieć nie rozdielałabym go.